In [26]:
import importlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib._tri import Triangulation
import NN
importlib.reload(NN)
from dvclive import Live

In [27]:
layers = [
    NN.DenseLayer(2, 0, NN.dummy_activation, NN.dummy_activation, True, False),
    NN.DenseLayer(4, 2, NN.sigmoid, NN.sigmoid_prime),
    NN.DenseLayer(2, 4, NN.sigmoid, NN.sigmoid_prime, False, True)
]
learning_rate = 0.01
lamda = 0.2

In [28]:
# with Live(save_dvc_exp=True) as live:
    # live.log_param("Learning Rate", learning_rate)
losses = []
for i in range(2000):
    # live.log_param('Epoch', i)
    a2 = layers[1].forward_propagation(X)
    loss = layers[2].loss(a2, y, 2, lamda)
    losses.append(loss)
    # live.log_metric("Loss", loss)
    NN.backward_propagate(layers, X, y, learning_rate, lamda)
    # live.next_step()

ValueError: operands could not be broadcast together with shapes (2,2) (2,5) 